In [1]:
!pip install --upgrade azure-cognitiveservices-vision-computervision
!pip install pillow


[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [40]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from msrest.authentication import CognitiveServicesCredentials
from array import array
import os
from PIL import Image
import sys
import time
import re
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np
from geopy.distance import geodesic
import folium
import holidays


In [ ]:
subscription_key = 'MCpv4Z6aiXGPnLUcXuRcSsN9CI126O7ka93O7eWw4stezHS5nzPfJQQJ99BEAC5RqLJXJ3w3AAAFACOGc3CM'
endpoint = 'https://rejsekortocrwesteurope.cognitiveservices.azure.com/'
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

In [14]:
# Local image path
image_path = r"C:\Users\gerga\OneDrive\Desktop\Rejsekort\Rejsekort 1.jpg"

# Open the file
with open(image_path, "rb") as image_stream:
    ocr_result = computervision_client.read_in_stream(image_stream, raw=True)

# Get the operation location (URL with an ID)
operation_location = ocr_result.headers["Operation-Location"]
operation_id = operation_location.split("/")[-1]


In [ ]:
while True:
    result = computervision_client.get_read_result(operation_id)
    if result.status not in ['notStarted', 'running']:
        break
    time.sleep(1)

# Parse the result
if result.status == "succeeded":
    for page in result.analyze_result.read_results:
        for line in page.lines:
            print(line.text)


11/3/24
09:19 Vestre Ringgade/Silkeborgvej (Aarhus Kom)
09:42 Skøjtehallen/Halmstadgade (Aarhus Kom)
DKK 18.00 >
11/2/24
20:00 Aarhus Tech/Dollerupvej (Aarhus Kom)
20:07 Klostertorvet (Aarhus Kom)
DKK 18.00 >


In [ ]:
source_folder = r"C:\Users\gerga\OneDrive\Desktop\Rejsekort"
output_data = []



for filename in os.listdir(source_folder):
    if filename.lower().endswith(".jpg"):
        filepath = os.path.join(source_folder, filename)
        print(f"📄 Processing: {filename}")

        # --- OCR with Azure ---
        with open(filepath, "rb") as image_stream:
            ocr_result = computervision_client.read_in_stream(image_stream, raw=True)

        operation_id = ocr_result.headers["Operation-Location"].split("/")[-1]

        while True:
            result = computervision_client.get_read_result(operation_id)
            if result.status not in ['notStarted', 'running']:
                break
            time.sleep(1)

        if result.status != "succeeded":
            print(f"OCR failed for {filename}")
            continue

        # --- Collect all lines of text ---
        lines = []
        for page in result.analyze_result.read_results:
            lines.extend([line.text.strip() for line in page.lines])

        current_date = None
        for i in range(len(lines) - 2):
            # Detect and update current date
            if re.match(r"\d{1,2}/\d{1,2}/\d{2,4}", lines[i]):
                current_date = datetime.strptime(lines[i], "%m/%d/%y").strftime("%Y-%m-%d")

            # Try parsing a journey from 3 lines: dep, arr, fare
            dep_match = re.match(r"(\d{2}:\d{2})\s+(.*)", lines[i])
            arr_match = re.match(r"(\d{2}:\d{2})\s+(.*)", lines[i + 1])
            fare_match = re.search(r"DKK\s+([0-9]+[.,][0-9]{2})", lines[i + 2])

            if dep_match and arr_match and fare_match:
                from_time = dep_match.group(1)
                from_stop = dep_match.group(2).strip()
                to_time = arr_match.group(1)
                to_stop = arr_match.group(2).strip()
                fare = float(fare_match.group(1).replace(",", "."))

                output_data.append({
                    "Date": current_date,
                    "Departure Time": from_time,
                    "Departure Stop": from_stop,
                    "Arrival Time": to_time,
                    "Arrival Stop": to_stop,
                    "Fare DKK": fare,
                    "Filename": filename
                })

        time.sleep(3)  # Wait 3 seconds before processing the next file


# === Save all extracted data to CSV ===
df = pd.DataFrame(output_data)
df.to_csv("rejsekort_jpg_extracted.csv", index=False)
print("Finished: All data saved to 'rejsekort_jpg_extracted.csv'")

📄 Processing: Rejsekort 1.jpg
📄 Processing: Rejsekort 10.jpg
📄 Processing: Rejsekort 11.jpg
📄 Processing: Rejsekort 12.jpg
📄 Processing: Rejsekort 13.jpg
📄 Processing: Rejsekort 14.jpg
📄 Processing: Rejsekort 15.jpg
📄 Processing: Rejsekort 16.jpg
📄 Processing: Rejsekort 17.jpg
📄 Processing: Rejsekort 18.jpg
📄 Processing: Rejsekort 19.jpg
📄 Processing: Rejsekort 2.jpg
📄 Processing: Rejsekort 20.jpg
📄 Processing: Rejsekort 21.jpg
📄 Processing: Rejsekort 22.jpg
📄 Processing: Rejsekort 23.jpg
📄 Processing: Rejsekort 24.jpg
📄 Processing: Rejsekort 25.jpg
📄 Processing: Rejsekort 26.jpg
📄 Processing: Rejsekort 27.jpg
📄 Processing: Rejsekort 28.jpg
📄 Processing: Rejsekort 29.jpg
📄 Processing: Rejsekort 3.jpg
📄 Processing: Rejsekort 30.jpg
📄 Processing: Rejsekort 31.jpg
📄 Processing: Rejsekort 32.jpg
📄 Processing: Rejsekort 33.jpg
📄 Processing: Rejsekort 34.jpg
📄 Processing: Rejsekort 35.jpg
📄 Processing: Rejsekort 36.jpg
📄 Processing: Rejsekort 37.jpg
📄 Processing: Rejsekort 38.jpg
📄 Processin